互相关运算

严格来说，卷积层是个错误的叫法，因为它所表达的运算其实是互相关运算（cross-correlation），而不是卷积运算。 

首先考虑较为简单的情况，也就是忽略通道，仅关注如何处理二维图像数据和隐藏表示

在使用卷积核卷积后，输出大小略小于输入大小。这是因为卷积核的宽度和高度大于1， 而卷积核只与图像中每个大小完全适合的位置进行互相关运算。 所以，输出大小等于输入大小$n_h \times n_w$减去卷积核大小$k_h \times k_w$，即$(n_h-k_h+1) \times (n_w-k_w+1)$

因为要在足够的空间上移动卷积核，因此在每次输出后需要在图像边界周围填充零来保证有足够的空间移动卷积核，从而保持输出大小不变

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l

# X表示输入张量，K表示卷积核张量

def corr2d(X, K):  #@save
    """计算二维互相关运算"""
    h, w = K.shape

    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))  # 输出的大小
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

通过示例来进行检验

In [2]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

卷积层

卷积层对输入和卷积核权重进行互相关运算，并在添加标量偏置后产生输出

于是卷积层中的两个被训练的参数是卷积核权重和标量偏置。 就像我们之前随机初始化全连接层一样，在训练基于卷积层的模型时，我们也随机初始化卷积核权重。

在__init__构造函数中，将weight和bias声明为两个模型参数。前向传播函数调用corr2d函数并添加偏置。

In [3]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

使用卷积层来实现图像中目标的边缘检测

In [6]:
# 通过找到像素变化的位置来检测图像中不同颜色的边缘

# 构造6*8像素的黑白图像
X = torch.ones((6, 8))
X[:, 2:6] = 0
print(X)

# 构造一个宽为2，高为1的卷积核
K = torch.tensor([[1.0, -1.0]])
print(K)

# 此时使用卷积核进行互相关运算，得到结果，输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘，其他情况的输出为0
Y = corr2d(X, K)
Y

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])
tensor([[ 1., -1.]])


tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

因为我们设置的卷积核是宽为2，高为1的
因此只能检测垂直边缘，无法检测水平边缘

In [7]:
# 将输入特征X转置后就无法得到边缘

corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

以上是手动设计的卷积核，但面对更复杂的情况，显然需要学习卷积核

我们先构造一个卷积层，并将其卷积核初始化为随机张量。接下来，在每次迭代中，我们比较Y与卷积层输出的平方误差，然后计算梯度来更新卷积核。为了简单起见，我们在此使用内置的二维卷积层，并忽略偏置。

In [8]:
# 构造一个二维卷积层，它具有1个输入通道和1个输出通道，卷积核的大小是（1，2）
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 输入图像和输出标签定义为四维张量以符合PyTorch卷积层的输入格式
# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

lr = 3e-2  # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2 # 损失函数为均方误差
    conv2d.zero_grad() # 清除上一步的梯度，准备进行这一轮的反向传播
    l.sum().backward() # 计算损失函数关于卷积核参数的梯度
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 2.032
epoch 4, loss 0.429
epoch 6, loss 0.108
epoch 8, loss 0.033
epoch 10, loss 0.012


通过下面的代码来查看所学卷积核的权重张量

In [9]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9828, -1.0037]])

为了与深度学习文献中的标准术语保持一致，我们将继续把“互相关运算”称为卷积运算，尽管严格地说，它们略有不同。 此外，对于卷积核张量上的权重，我们称其为元素。

输出的卷积层有时被称为特征映射（feature map），因为它可以被视为一个输入映射到下一层的空间维度的转换器。 在卷积神经网络中，对于某一层的任意元素x，其感受野（receptive field）是指在前向传播期间可能影响x计算的所有元素（来自所有先前层）。